# 1.选择文件夹

In [1]:
timing = input('am or pm: ')

am or pm: pm


In [2]:
import pandas as pd
import os

In [3]:
list_file = list(os.walk('./'))[0][2]
print(list_file)

for file in list_file:
    if 'MRP' in file:
        mrp = './' + file
    elif '断货改价' in file:
        last = './' + file
    elif '改价表' in file:
        stcok = './' + file 
    elif '替代ok' in file:
        alternate = './' + file
    elif '替代update' in file:
        update = './' + file
    elif '替代料总表' in file:
        combination = './' + file
    elif '每日品类' in file:
        day_category = './' + file

print(f'MRP \t{mrp}')
print(f'last \t{last}')
print(f'stock \t{stcok}')
print(f'alternate \t{alternate}')
print(f'update \t{update}')
print(f'combination \t{combination}')
print(f'day_category \t{day_category}')

['1.替代料表整理-Bear20230724.ipynb', 'US MRP20231127-运营.xlsx', '大账号 断货改价 上午 12.4.xlsx', '大账号_(Attention) _3.ipynb', '改价表更新 12-4.xlsx', '改价记录提取最新.ipynb', '替代ok表_20231129_180633.xlsx', '替代update表_20231129_175939.xlsx', '替代料总表11-27.xlsx', '每日品类调价 9.27.xls']
MRP 	./US MRP20231127-运营.xlsx
last 	./大账号 断货改价 上午 12.4.xlsx
stock 	./改价表更新 12-4.xlsx
alternate 	./替代ok表_20231129_180633.xlsx
update 	./替代update表_20231129_175939.xlsx
combination 	./替代料总表11-27.xlsx
day_category 	./每日品类调价 9.27.xls


# 2.替代料表

In [4]:
df_alternate = pd.read_excel(alternate).fillna('')
df_alternate['row'] = [str(i) for i in range(len(df_alternate))]
df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,row
0,USCA171876D,USCA171876-3,USCA171876D-3,USCA171876,,,,0
1,USWL02211,USWL02211H,,,,,,1
2,USTHA7040PX,USTHA7040P,,,,,,2
3,USBMR3204K,USBMR3204Y,,,,,,3
4,USDLA46848G,USDLA46848,,,,,,4
...,...,...,...,...,...,...,...,...
46298,USWS02353,USWS02353-C,USWS02353U-C,,,,,46298
46299,USIC02205L-6,USIC02205A-6,USIC02205I-6,USIC02205L-6-C,USIC02205I-6-C,,,46299
46300,USTSS18440-C,USTSS18440U-C,,,,,,46300
46301,USHS55595-C,USHS55595N-C,USHS55595,,,,,46301


# 3. 计算在途总数、库存

In [5]:
# 导入改价表数据计算在途总数以及每个SKU的库存
df_stock = pd.read_excel(stcok,
                         names=['SKU', '在途总数', 'YKD库存'],
                         usecols='A,E,F'
                         ).fillna(0)
df_stock

,SKU,在途总数,YKD库存
0,USOP64305,0.0,3
1,USOP25053,0.0,0
2,USSGPL002,2.0,0
3,USOP97704,7.0,0
4,USOP97705,5.0,0
...,...,...,...
293525,USIM15178-C,0.0,1
293526,USACC67232-C,0.0,1
293527,USTBC00291,0.0,1
293528,USCVA13330-2-C,0.0,1


In [6]:
# 计算自身以及-N以及-NC的库存
df_stock['SKU'] = [df_stock.loc[i, 'SKU'][:-2] if df_stock.loc[i, 'SKU'].endswith('-N') else df_stock.loc[i, 'SKU'][:-3] if df_stock.loc[i, 'SKU'].endswith('-NC') else df_stock.loc[i,'SKU'] for i in range(len(df_stock))]

# 汇总每个SKU的库存
df_stock = df_stock.groupby(df_stock['SKU'], as_index=False).sum()
df_stock

,SKU,在途总数,YKD库存
0,BPD8550086500+BRS31318,0.0,1
1,CAFP9183M0,0.0,1
2,CAWR41163,0.0,1
3,CS20230609,0.0,0
4,CS230530,0.0,0
...,...,...,...
90324,YHTWS58202-Y-6,0.0,0
90325,YHTWS5W1ZBA-Y-10,0.0,0
90326,YHTWS5W1ZBA-Y-8,0.0,0
90327,YHTWS802AA-Y-6,0.0,0


In [7]:
# 得到每个替代料的库存数量
df_alternate['在途总数'] = 0
df_alternate['YKD库存'] = 0

df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='主料SKU', right_on='SKU', suffixes=('','_0')).fillna(0)
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料1', right_on='SKU', suffixes=('','_1')).fillna(0)
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料2', right_on='SKU', suffixes=('','_2')).fillna(0)
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料3', right_on='SKU', suffixes=('','_3')).fillna(0)
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料4', right_on='SKU', suffixes=('','_4')).fillna(0)
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料5', right_on='SKU', suffixes=('','_5')).fillna(0)
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料6', right_on='SKU', suffixes=('','_6')).fillna(0)
df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,row,在途总数,YKD库存,...,YKD库存_3,SKU_4,在途总数_4,YKD库存_4,SKU_5,在途总数_5,YKD库存_5,SKU_6,在途总数_6,YKD库存_6
0,USCA171876D,USCA171876-3,USCA171876D-3,USCA171876,,,,0,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
1,USWL02211,USWL02211H,,,,,,1,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
2,USTHA7040PX,USTHA7040P,,,,,,2,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
3,USBMR3204K,USBMR3204Y,,,,,,3,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
4,USDLA46848G,USDLA46848,,,,,,4,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46298,USWS02353,USWS02353-C,USWS02353U-C,,,,,46298,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
46299,USIC02205L-6,USIC02205A-6,USIC02205I-6,USIC02205L-6-C,USIC02205I-6-C,,,46299,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
46300,USTSS18440-C,USTSS18440U-C,,,,,,46300,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
46301,USHS55595-C,USHS55595N-C,USHS55595,,,,,46301,0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0


In [8]:
df_alternate.columns

Index(['主料SKU', '替换料1', '替换料2', '替换料3', '替换料4', '替换料5', '替换料6', 'row', '在途总数',
       'YKD库存', 'SKU', '在途总数_0', 'YKD库存_0', 'SKU_1', '在途总数_1', 'YKD库存_1',
       'SKU_2', '在途总数_2', 'YKD库存_2', 'SKU_3', '在途总数_3', 'YKD库存_3', 'SKU_4',
       '在途总数_4', 'YKD库存_4', 'SKU_5', '在途总数_5', 'YKD库存_5', 'SKU_6', '在途总数_6',
       'YKD库存_6'],
      dtype='object')

In [9]:
# 计算每个替代料的“在途总数”以及”替代料库存“
df_alternate['在途总数'] = df_alternate['在途总数_0'] + df_alternate['在途总数_1'] + df_alternate['在途总数_2'] + df_alternate['在途总数_3'] + df_alternate['在途总数_4'] + df_alternate['在途总数_5'] + df_alternate['在途总数_6'] 

df_alternate['YKD库存'] = df_alternate['YKD库存_0'] + df_alternate['YKD库存_1'] + df_alternate['YKD库存_2'] + df_alternate['YKD库存_3'] + df_alternate['YKD库存_4'] + df_alternate['YKD库存_5'] + df_alternate['YKD库存_6'] 

df_alternate = df_alternate[['主料SKU',
                           '替换料1',
                           '替换料2',
                           '替换料3',
                           '替换料4',
                           '替换料5',
                           '替换料6',
                           '在途总数',
                           'YKD库存',
                           'row']].copy()
df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,在途总数,YKD库存,row
0,USCA171876D,USCA171876-3,USCA171876D-3,USCA171876,,,,2.0,8.0,0
1,USWL02211,USWL02211H,,,,,,86.0,14.0,1
2,USTHA7040PX,USTHA7040P,,,,,,3.0,21.0,2
3,USBMR3204K,USBMR3204Y,,,,,,23.0,14.0,3
4,USDLA46848G,USDLA46848,,,,,,0.0,15.0,4
...,...,...,...,...,...,...,...,...,...,...
46298,USWS02353,USWS02353-C,USWS02353U-C,,,,,0.0,7.0,46298
46299,USIC02205L-6,USIC02205A-6,USIC02205I-6,USIC02205L-6-C,USIC02205I-6-C,,,0.0,1.0,46299
46300,USTSS18440-C,USTSS18440U-C,,,,,,0.0,1.0,46300
46301,USHS55595-C,USHS55595N-C,USHS55595,,,,,0.0,21.0,46301


In [10]:
list_tmp = []

for i in df_alternate.columns[:-3]:
    df_tmp = df_alternate[[i,
                          '在途总数',
                          'YKD库存',
                          'row']]
    df_tmp.columns = ['SKU','在途总数', 'YKD库存', 'row']
    df_tmp = df_tmp[df_tmp['SKU'] != '']
    list_tmp.append(df_tmp)
    
df_position = pd.concat(list_tmp).reset_index(drop=True)
df_position

,SKU,在途总数,YKD库存,row
0,USCA171876D,2.0,8.0,0
1,USWL02211,86.0,14.0,1
2,USTHA7040PX,3.0,21.0,2
3,USBMR3204K,23.0,14.0,3
4,USDLA46848G,0.0,15.0,4
...,...,...,...,...
99805,USGS6310T,25.0,2.0,20777
99806,USIC2A367A,5.0,2.0,34464
99807,USIC04470,9.0,4.0,34562
99808,USIC02680I-6,11.0,42.0,38636


In [11]:
df_stock = pd.concat([df_position,df_stock]).drop_duplicates(['SKU'], keep='first').reset_index(drop=True).fillna('')
df_stock

,SKU,在途总数,YKD库存,row
0,USCA171876D,2.0,8.0,0
1,USWL02211,86.0,14.0,1
2,USTHA7040PX,3.0,21.0,2
3,USBMR3204K,23.0,14.0,3
4,USDLA46848G,0.0,15.0,4
...,...,...,...,...
123180,YHTWS58202-Y-6,0.0,0.0,
123181,YHTWS5W1ZBA-Y-10,0.0,0.0,
123182,YHTWS5W1ZBA-Y-8,0.0,0.0,
123183,YHTWS802AA-Y-6,0.0,0.0,


In [12]:
df_stock_way = df_stock[['SKU', '在途总数']]
df_stock_way

,SKU,在途总数
0,USCA171876D,2.0
1,USWL02211,86.0
2,USTHA7040PX,3.0
3,USBMR3204K,23.0
4,USDLA46848G,0.0
...,...,...
123180,YHTWS58202-Y-6,0.0
123181,YHTWS5W1ZBA-Y-10,0.0
123182,YHTWS5W1ZBA-Y-8,0.0
123183,YHTWS802AA-Y-6,0.0


In [13]:
df_stock = df_stock[['SKU', 'YKD库存', 'row']].copy()
df_stock['总库存'] = df_stock['YKD库存']
df_stock.loc[:,'组合料库存'] = 0
df_stock = df_stock[['SKU',
                     '组合料库存',
                     'YKD库存',
                     '总库存',
                     'row']]
df_stock

,SKU,组合料库存,YKD库存,总库存,row
0,USCA171876D,0,8.0,8.0,0
1,USWL02211,0,14.0,14.0,1
2,USTHA7040PX,0,21.0,21.0,2
3,USBMR3204K,0,14.0,14.0,3
4,USDLA46848G,0,15.0,15.0,4
...,...,...,...,...,...
123180,YHTWS58202-Y-6,0,0.0,0.0,
123181,YHTWS5W1ZBA-Y-10,0,0.0,0.0,
123182,YHTWS5W1ZBA-Y-8,0,0.0,0.0,
123183,YHTWS802AA-Y-6,0,0.0,0.0,


# 3.2 组合料库存

In [14]:
# 从替代料总表中读取组合料关系，去除数量为0，为1，为空的情况
df_combination = pd.read_excel(combination,
                               header=1,
                               dtype={'数量':str})[['主料SKU','数量','SKU1','SKU2','SKU3','SKU4','SKU5']].fillna('')

df_combination = df_combination[(df_combination['数量'] != '0') & (df_combination['数量'] != '1') & (df_combination['数量'] != '') & (df_combination['数量'] != '未更新')].reset_index(drop=True)

df_combination

,主料SKU,数量,SKU1,SKU2,SKU3,SKU4,SKU5
0,USWCP52105-2,1+1,USWCP02525,USWCP05210,,,
1,USCA51317DLR,1+1,USCA51317D,USCA51318D,,,
2,USBCB84615LR,1+1,USBCB84615,USBCB84614,,,
3,USCA00103D-2,2,USCA00103D,,,,
4,USCA80215D-2,2,USCA80215D,,,,
...,...,...,...,...,...,...,...
4964,USBCB70607LR-C,1+1,USBCB92706-C,USBCB92707-C,,,
4965,USBCN11617LR-C,1+1,USBCN84116-C,USBCN84117-C,,,
4966,USGS5B870LR-C,1+1,USGS5B870L-1-C,USGS5B870R-1-C,,,
4967,USBCB93233LR-C,1+1,USBCB97532-C,USBCB97533-C,,,


In [15]:
def DEduplicate(df):
    list_row = list(set(df['row'].tolist()))
    if '' in list_row:
        list_row.remove('')
    list_row.sort()
    
    dict_row = {}
    for row in list_row:
        dict_row[row] = ''
    for i in range(len(df)):
        if df.loc[i, 'row'] != '':
            dict_row[df.loc[i, 'row']] += str(i) + ';'
    
    list_duplicated = []
    for i in dict_row:
        list_row = dict_row[i].split(';')[:-1]
        list_row.sort()
        length = len(list_row)
        if length > 1:
            for j in range(length -1):
                list_duplicated.append(list_row[j])
                
    for i in list_duplicated:
        df = df.drop(int(i))
        
    return df.reset_index(drop=True),list_duplicated

In [16]:
# 左连接获得替代料关系原来的行位置，如果有重复则删除
df_combination = pd.merge(left=df_combination, right=df_position, how='left', left_on='主料SKU', right_on='SKU', suffixes=('','_0')).fillna('')
df_combination,l = DEduplicate(df_combination)

print(list(df_combination.columns))
df_combination

['主料SKU', '数量', 'SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5', 'SKU', '在途总数', 'YKD库存', 'row']


,主料SKU,数量,SKU1,SKU2,SKU3,SKU4,SKU5,SKU,在途总数,YKD库存,row
0,USWCP52105-2,1+1,USWCP02525,USWCP05210,,,,,,,
1,USCA51317DLR,1+1,USCA51317D,USCA51318D,,,,USCA51317DLR,5.0,0.0,9
2,USBCB84615LR,1+1,USBCB84615,USBCB84614,,,,,,,
3,USCA00103D-2,2,USCA00103D,,,,,USCA00103D-2,2.0,0.0,13
4,USCA80215D-2,2,USCA80215D,,,,,USCA80215D-2,0.0,2.0,24
...,...,...,...,...,...,...,...,...,...,...,...
4964,USBCB70607LR-C,1+1,USBCB92706-C,USBCB92707-C,,,,,,,
4965,USBCN11617LR-C,1+1,USBCN84116-C,USBCN84117-C,,,,,,,
4966,USGS5B870LR-C,1+1,USGS5B870L-1-C,USGS5B870R-1-C,,,,,,,
4967,USBCB93233LR-C,1+1,USBCB97532-C,USBCB97533-C,,,,,,,


In [17]:
# 获得组合料每个SKU对应的库存
df_combination = pd.merge(left=df_combination, right=df_stock, how='left', left_on='SKU1', right_on='SKU', suffixes=('','_1')).fillna({'YKD库存_1':0})
df_combination = pd.merge(left=df_combination, right=df_stock, how='left', left_on='SKU2', right_on='SKU', suffixes=('','_2')).fillna({'YKD库存_2':0})
df_combination = pd.merge(left=df_combination, right=df_stock, how='left', left_on='SKU3', right_on='SKU', suffixes=('','_3')).fillna({'YKD库存_3':0})
df_combination = pd.merge(left=df_combination, right=df_stock, how='left', left_on='SKU4', right_on='SKU', suffixes=('','_4')).fillna({'YKD库存_4':0})
df_combination = pd.merge(left=df_combination, right=df_stock, how='left', left_on='SKU5', right_on='SKU', suffixes=('','_5')).fillna({'YKD库存_5':0})
df_combination

,主料SKU,数量,SKU1,SKU2,SKU3,SKU4,SKU5,SKU,在途总数,YKD库存,...,SKU_4,组合料库存_4,YKD库存_4,总库存_4,row_4,SKU_5,组合料库存_5,YKD库存_5,总库存_5,row_5
0,USWCP52105-2,1+1,USWCP02525,USWCP05210,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,USCA51317DLR,1+1,USCA51317D,USCA51318D,,,,USCA51317DLR,5.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,USBCB84615LR,1+1,USBCB84615,USBCB84614,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,USCA00103D-2,2,USCA00103D,,,,,USCA00103D-2,2.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4,USCA80215D-2,2,USCA80215D,,,,,USCA80215D-2,0.0,2.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4964,USBCB70607LR-C,1+1,USBCB92706-C,USBCB92707-C,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4965,USBCN11617LR-C,1+1,USBCN84116-C,USBCN84117-C,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4966,USGS5B870LR-C,1+1,USGS5B870L-1-C,USGS5B870R-1-C,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4967,USBCB93233LR-C,1+1,USBCB97532-C,USBCB97533-C,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [18]:
df_combination.columns

Index(['主料SKU', '数量', 'SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5', 'SKU', '在途总数',
       'YKD库存', 'row', 'SKU_1', '组合料库存', 'YKD库存_1', '总库存', 'row_1', 'SKU_2',
       '组合料库存_2', 'YKD库存_2', '总库存_2', 'row_2', 'SKU_3', '组合料库存_3', 'YKD库存_3',
       '总库存_3', 'row_3', 'SKU_4', '组合料库存_4', 'YKD库存_4', '总库存_4', 'row_4',
       'SKU_5', '组合料库存_5', 'YKD库存_5', '总库存_5', 'row_5'],
      dtype='object')

In [19]:
# 计算组合料库存，根据“数量”关系来计算
import math

df_combination['组合料库存'] = 0
for i in range(len(df_combination)):
    list_amount_number = df_combination.loc[i, '数量'].split('+')
    list_amount_stock = []
    for j in range(len(list_amount_number)):
        list_amount_stock.append(df_combination.loc[i, 'YKD库存_'+str(j+1)] / int(list_amount_number[j]))
    df_combination.loc[i, '组合料库存'] = math.floor(min(list_amount_stock))
    
print(list(df_combination.columns))
df_combination

['主料SKU', '数量', 'SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5', 'SKU', '在途总数', 'YKD库存', 'row', 'SKU_1', '组合料库存', 'YKD库存_1', '总库存', 'row_1', 'SKU_2', '组合料库存_2', 'YKD库存_2', '总库存_2', 'row_2', 'SKU_3', '组合料库存_3', 'YKD库存_3', '总库存_3', 'row_3', 'SKU_4', '组合料库存_4', 'YKD库存_4', '总库存_4', 'row_4', 'SKU_5', '组合料库存_5', 'YKD库存_5', '总库存_5', 'row_5']


,主料SKU,数量,SKU1,SKU2,SKU3,SKU4,SKU5,SKU,在途总数,YKD库存,...,SKU_4,组合料库存_4,YKD库存_4,总库存_4,row_4,SKU_5,组合料库存_5,YKD库存_5,总库存_5,row_5
0,USWCP52105-2,1+1,USWCP02525,USWCP05210,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,USCA51317DLR,1+1,USCA51317D,USCA51318D,,,,USCA51317DLR,5.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,USBCB84615LR,1+1,USBCB84615,USBCB84614,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,USCA00103D-2,2,USCA00103D,,,,,USCA00103D-2,2.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4,USCA80215D-2,2,USCA80215D,,,,,USCA80215D-2,0.0,2.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4964,USBCB70607LR-C,1+1,USBCB92706-C,USBCB92707-C,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4965,USBCN11617LR-C,1+1,USBCN84116-C,USBCN84117-C,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4966,USGS5B870LR-C,1+1,USGS5B870L-1-C,USGS5B870R-1-C,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4967,USBCB93233LR-C,1+1,USBCB97532-C,USBCB97533-C,,,,,,,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [20]:
df_combination = df_combination[['主料SKU',
                                 '组合料库存',
                                ]].copy()
df_combination = pd.merge(left=df_combination, right=df_position, how='left', left_on='主料SKU', right_on='SKU').fillna({'YKD库存':0, '在途总数':0,  'row':''})
df_combination

,主料SKU,组合料库存,SKU,在途总数,YKD库存,row
0,USWCP52105-2,0,NaN,0.0,0.0,
1,USCA51317DLR,0,USCA51317DLR,5.0,0.0,9
2,USBCB84615LR,1,NaN,0.0,0.0,
3,USCA00103D-2,0,USCA00103D-2,2.0,0.0,13
4,USCA80215D-2,0,USCA80215D-2,0.0,2.0,24
...,...,...,...,...,...,...
4964,USBCB70607LR-C,0,NaN,0.0,0.0,
4965,USBCN11617LR-C,0,NaN,0.0,0.0,
4966,USGS5B870LR-C,0,NaN,0.0,0.0,
4967,USBCB93233LR-C,0,NaN,0.0,0.0,


In [21]:
# 根据辅助列row来区分是否有重复的替代料关系需要合并计算库存，所以要区分开row为空以及不为空的情况
df_combination_1 = df_combination[df_combination.loc[:, 'row']==''].copy()
df_combination_1['总库存'] = df_combination_1.loc[:, '组合料库存'] + df_combination_1.loc[:, 'YKD库存']
df_combination_1 = df_combination_1[['主料SKU',
                                     '组合料库存',
                                     'YKD库存',
                                     '总库存',
                                     'row']].reset_index(drop=True).copy()

df_combination_1.columns = ['SKU', '组合料库存','YKD库存','总库存', 'row']
df_combination_1

,SKU,组合料库存,YKD库存,总库存,row
0,USWCP52105-2,0,0.0,0.0,
1,USBCB84615LR,1,0.0,1.0,
2,USDAV49320-4,1,0.0,1.0,
3,USDAV893EA-4,5,0.0,5.0,
4,USDLA46184LR,1,0.0,1.0,
...,...,...,...,...,...
2368,USBCB70607LR-C,0,0.0,0.0,
2369,USBCN11617LR-C,0,0.0,0.0,
2370,USGS5B870LR-C,0,0.0,0.0,
2371,USBCB93233LR-C,0,0.0,0.0,


In [22]:
def custom_agg(row_column):
    # 检查是否有字符串或空值，如果有，返回第一个非空字符串，否则返回空字符串
    non_empty_strings = row_column.dropna().astype(str)
    if len(non_empty_strings) > 0:
        return non_empty_strings.iloc[0]
    else:
        return ''

In [23]:
df_s = pd.concat([df_stock, df_combination_1]).reset_index(drop=True).fillna('')

df_s = df_s.groupby(df_s['SKU'], as_index=False).agg({
     '组合料库存':'sum',
     'YKD库存':'sum',
     '总库存': 'sum',             
     'row': custom_agg             
})
df_s

,SKU,组合料库存,YKD库存,总库存,row
0,BPD8550086500+BRS31318,0,1.0,1.0,
1,CAFP9183M0,0,1.0,1.0,
2,CAWR41163,0,1.0,1.0,
3,CS20230609,0,0.0,0.0,
4,CS230530,0,0.0,0.0,
...,...,...,...,...,...
123181,YHTWS58202-Y-6,0,0.0,0.0,
123182,YHTWS5W1ZBA-Y-10,0,0.0,0.0,
123183,YHTWS5W1ZBA-Y-8,0,0.0,0.0,
123184,YHTWS802AA-Y-6,0,0.0,0.0,


In [24]:
df_combination_2 = df_combination[df_combination.loc[:, 'row']!=''][['组合料库存',
                                                                    'row']]
df_combination_2.reset_index(drop=True)

,组合料库存,row
0,0,9
1,0,13
2,0,24
3,2,27
4,0,42
...,...,...
2591,1,46287
2592,3,46289
2593,0,46290
2594,0,46292


In [25]:
df_s = pd.merge(left=df_s, right=df_combination_2, how='left', left_on='row', right_on='row', suffixes=('','_x')).fillna(0)
df_s['总库存'] = df_s.loc[:, 'YKD库存'] + df_s.loc[:, '组合料库存'] + df_s.loc[:, '组合料库存_x']
df_s['组合料库存'] = df_s.loc[:, '组合料库存'] + df_s.loc[:, '组合料库存_x']
df_s

,SKU,组合料库存,YKD库存,总库存,row,组合料库存_x
0,BPD8550086500+BRS31318,0.0,1.0,1.0,,0.0
1,CAFP9183M0,0.0,1.0,1.0,,0.0
2,CAWR41163,0.0,1.0,1.0,,0.0
3,CS20230609,0.0,0.0,0.0,,0.0
4,CS230530,0.0,0.0,0.0,,0.0
...,...,...,...,...,...,...
123181,YHTWS58202-Y-6,0.0,0.0,0.0,,0.0
123182,YHTWS5W1ZBA-Y-10,0.0,0.0,0.0,,0.0
123183,YHTWS5W1ZBA-Y-8,0.0,0.0,0.0,,0.0
123184,YHTWS802AA-Y-6,0.0,0.0,0.0,,0.0


In [26]:
df_s = df_s[['SKU',
             '组合料库存',
             'YKD库存',
             '总库存']].copy()
df_stock = df_s
df_stock

,SKU,组合料库存,YKD库存,总库存
0,BPD8550086500+BRS31318,0.0,1.0,1.0
1,CAFP9183M0,0.0,1.0,1.0
2,CAWR41163,0.0,1.0,1.0
3,CS20230609,0.0,0.0,0.0
4,CS230530,0.0,0.0,0.0
...,...,...,...,...
123181,YHTWS58202-Y-6,0.0,0.0,0.0
123182,YHTWS5W1ZBA-Y-10,0.0,0.0,0.0
123183,YHTWS5W1ZBA-Y-8,0.0,0.0,0.0
123184,YHTWS802AA-Y-6,0.0,0.0,0.0


# 4.MRP表整理

In [27]:
# 导入MRP数据，整合后续的断货改价表
df_mrp = pd.read_excel(mrp,
                       header=1,
                       names=['SKU','品类','层级','身份','负责人'],
                       usecols='B,G,H,J,AR')
df_mrp

,SKU,品类,层级,身份,负责人
0,USTCV17215HIE-2-C,TCV,TOP1,老品,Ada
1,USOBDTK100,OBD,TOP10,老品,Ada
2,USBDA04108U-2-C,BDA,TOP1,老品,Ada
3,USET87039H-C,ET,TOP1,老品,Ada
4,USGS41420B-C,GS,TOP10,老品,Ada
...,...,...,...,...,...
63632,USIC677650-8-C,IC,TOP10,老品,Rosie
63633,USDS86070-C,DS,TOP20,新品,Rosie
63634,USWR41521S-C1,WR,普通,老品,Panki
63635,USSGUN005-2R-C,SG,普通,断货,Panki


In [28]:
# 筛选除CA品类以及CA-的品类都是Hebe负责的
import pandas as pd

hebe = ['CA' if category[:3] == 'CA-' else category if category=='CA' else category for category in df_mrp.loc[:, '品类']]

hebe

['TCV',
 'OBD',
 'BDA',
 'ET',
 'GS',
 'ICK',
 'BMR',
 'GS',
 'GS',
 'TBC',
 'ICK',
 'TPMS',
 'STR',
 'EVC',
 'ABS',
 'BDA',
 'IM',
 'BDA',
 'BT',
 'GS',
 'IC',
 'IC',
 'SG',
 'WR',
 'ABS',
 'CVA',
 'CCS',
 'FP',
 'TSS',
 'WS',
 'ICK',
 'WIM',
 'PSS',
 'ICK',
 'STR',
 'RAD',
 'CAF',
 'GS',
 'BDA',
 'DS',
 'SRM',
 'THA',
 'HB',
 'THK',
 'WHS',
 'TPMS',
 'ALT',
 'ICK',
 'BDA',
 'DLA',
 'ET',
 'GS',
 'GS',
 'GS',
 'GS',
 'HB',
 'IC',
 'IM',
 'OFA',
 'OP',
 'OS',
 'SG',
 'WIM',
 'HB',
 'ICK',
 'BC',
 'TPMS',
 'ICK',
 'EAF',
 'GS',
 'BC',
 'BDA',
 'BMR',
 'DIS',
 'DLA',
 'DS',
 'GS',
 'GS',
 'SG',
 'SG',
 'WP',
 'WR',
 'WR',
 'WR',
 'ABS',
 'ABS',
 'TPMS',
 'KS',
 'EMD',
 'CA',
 'EAF',
 'ABS',
 'ABS',
 'BDA',
 'BM',
 'CA',
 'FP',
 'FP',
 'GS',
 'GS',
 'GS',
 'OP',
 'OS',
 'PSP',
 'THB',
 'TSS',
 'WP',
 'WR',
 'HBZ',
 'CVA',
 'PBB',
 'KS',
 'ID',
 'STR',
 'ICK',
 'RAD',
 'SPW',
 'VLT',
 'FI',
 'HVCP',
 'ICK',
 'ET',
 'BMR',
 'CAMK',
 'EAF',
 'EAF',
 'SP',
 'WS',
 'CPS',
 'BC',
 'BC',
 'BC',


In [29]:
df_m = df_mrp
df_m['品类'] = hebe
df_m[df_m.loc[:, '品类'] == 'CA']

,SKU,品类,层级,身份,负责人
89,USCAK11441U-C,CA,TOP10,老品,Hebe
95,USCA7L021D,CA,普通,断货,Hebe
183,USCAK21064-C,CA,TOP10,老品,Hebe
194,USCAKAU018D,CA,TOP1,老品,Hebe
404,USCAK41644-C,CA,TOP1,老品,Hebe
...,...,...,...,...,...
63347,USCA20474LR-C,CA,普通,老品,Hebe
63348,USCA40286LR-C,CA,普通,断货,Hebe
63352,USCA20024LR-C,CA,普通,老品,Hebe
63354,USCA01023334-4-C,CA,TOP20,老品,Hebe


In [30]:
# 筛选Hebe负责的品类
df_m = df_m[(df_m.loc[:,'负责人']=='Hebe') | (df_m.loc[:, '品类']=='CA')].reset_index(drop=True)
df_m

,SKU,品类,层级,身份,负责人
0,USABS18740H-C,ABS,TOP1,老品,Hebe
1,USABS44600-C,ABS,TOP10,断货,Hebe
2,USHB70308-C,HB,TOP1,老品,Hebe
3,USWHS11005-20-C,WHS,TOP1,老品,Hebe
4,USHB13214-C,HB,TOP10,老品,Hebe
...,...,...,...,...,...
26288,USBCB87273LR-C,BC,普通,新品,Hebe
26289,USABS00917-C,ABS,普通,断货,Hebe
26290,USABS2C204-C,ABS,普通,老品,Hebe
26291,USABS55539-C,ABS,普通,老品,Hebe


In [31]:
# 将负责人为NaN的转为Hebe
df_m['负责人'] = df_m['负责人'].map(lambda x : 'Hebe')

df_m

,SKU,品类,层级,身份,负责人
0,USABS18740H-C,ABS,TOP1,老品,Hebe
1,USABS44600-C,ABS,TOP10,断货,Hebe
2,USHB70308-C,HB,TOP1,老品,Hebe
3,USWHS11005-20-C,WHS,TOP1,老品,Hebe
4,USHB13214-C,HB,TOP10,老品,Hebe
...,...,...,...,...,...
26288,USBCB87273LR-C,BC,普通,新品,Hebe
26289,USABS00917-C,ABS,普通,断货,Hebe
26290,USABS2C204-C,ABS,普通,老品,Hebe
26291,USABS55539-C,ABS,普通,老品,Hebe


In [32]:
# 与position左连接，去重复行
df_m_position = pd.merge(left=df_m, right=df_position, how='left', left_on='SKU', right_on='SKU').fillna({'YKD库存':0, 'row':''}).fillna('')

print(list(df_m_position.columns))
df_m_position

['SKU', '品类', '层级', '身份', '负责人', '在途总数', 'YKD库存', 'row']


,SKU,品类,层级,身份,负责人,在途总数,YKD库存,row
0,USABS18740H-C,ABS,TOP1,老品,Hebe,26.0,4.0,36370
1,USABS44600-C,ABS,TOP10,断货,Hebe,16.0,0.0,43327
2,USHB70308-C,HB,TOP1,老品,Hebe,11.0,0.0,7082
3,USWHS11005-20-C,WHS,TOP1,老品,Hebe,21.0,41.0,45889
4,USHB13214-C,HB,TOP10,老品,Hebe,50.0,24.0,6317
...,...,...,...,...,...,...,...,...
26288,USBCB87273LR-C,BC,普通,新品,Hebe,,0.0,
26289,USABS00917-C,ABS,普通,断货,Hebe,19.0,5.0,45236
26290,USABS2C204-C,ABS,普通,老品,Hebe,8.0,8.0,45167
26291,USABS55539-C,ABS,普通,老品,Hebe,9.0,6.0,45282


In [33]:
# 获得MRP表的数据
df_m_position = pd.merge(left=df_m_position, right=df_stock, how='left', left_on='SKU', right_on='SKU', suffixes=('_position', '')).fillna(0)
df_m_position

,SKU,品类,层级,身份,负责人,在途总数,YKD库存_position,row,组合料库存,YKD库存,总库存
0,USABS18740H-C,ABS,TOP1,老品,Hebe,26.0,4.0,36370,0.0,4.0,4.0
1,USABS44600-C,ABS,TOP10,断货,Hebe,16.0,0.0,43327,0.0,0.0,0.0
2,USHB70308-C,HB,TOP1,老品,Hebe,11.0,0.0,7082,0.0,0.0,0.0
3,USWHS11005-20-C,WHS,TOP1,老品,Hebe,21.0,41.0,45889,0.0,41.0,41.0
4,USHB13214-C,HB,TOP10,老品,Hebe,50.0,24.0,6317,0.0,24.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...
26288,USBCB87273LR-C,BC,普通,新品,Hebe,,0.0,,0.0,0.0,0.0
26289,USABS00917-C,ABS,普通,断货,Hebe,19.0,5.0,45236,0.0,5.0,5.0
26290,USABS2C204-C,ABS,普通,老品,Hebe,8.0,8.0,45167,0.0,8.0,8.0
26291,USABS55539-C,ABS,普通,老品,Hebe,9.0,6.0,45282,0.0,6.0,6.0


In [34]:
df_mrp = df_m_position[['SKU', 
                        '品类',
                        '层级',
                        '身份',
                        'row']].copy()

df_mrp

,SKU,品类,层级,身份,row
0,USABS18740H-C,ABS,TOP1,老品,36370
1,USABS44600-C,ABS,TOP10,断货,43327
2,USHB70308-C,HB,TOP1,老品,7082
3,USWHS11005-20-C,WHS,TOP1,老品,45889
4,USHB13214-C,HB,TOP10,老品,6317
...,...,...,...,...,...
26288,USBCB87273LR-C,BC,普通,新品,
26289,USABS00917-C,ABS,普通,断货,45236
26290,USABS2C204-C,ABS,普通,老品,45167
26291,USABS55539-C,ABS,普通,老品,45282


# 5.基础整合

In [35]:
# df_last = pd.read_excel(last,
#                         header=1,
#                         usecols='C,D,U,V,X:AI,AK:AL,AO,AP,AS').fillna('')

df_last = pd.read_excel(last,
                        header=1,
                        usecols='B,C,I:V,X:Y,AB:AC').fillna('')

df_update = pd.read_excel(update,
                          header=0)

df_day_category = pd.read_excel(day_category,
                                names=['SKU', '调价记录'],
                                usecols='A,B').fillna('')

df_last['原始行号'] = [i+3 for i in range(len(df_last))]

print(df_last.columns)
df_last

Index([                  'SKU',                   '替代料',
                        '实时库存',                 '组合料库存',
                       'YKD库存',     2023-12-03 00:00:01,
           2023-12-01 00:00:01,     2023-11-30 00:00:01,
       '2023-12-03 00:00:01.1',     2023-12-02 00:00:01,
       '2023-12-01 00:00:01.1', '2023-11-30 00:00:01.1',
           2023-11-29 00:00:01,     2023-11-28 00:00:01,
           2023-11-27 00:00:01,     2023-11-26 00:00:01,
                        '改价记录',                  '断货时间',
                     '总品类调价记录',                '每日调价记录',
                        '原始行号'],
      dtype='object')


,SKU,替代料,实时库存,组合料库存,YKD库存,2023-12-03 00:00:01,2023-12-01 00:00:01,2023-11-30 00:00:01,2023-12-03 00:00:01.1,2023-12-02 00:00:01,...,2023-11-30 00:00:01.1,2023-11-29 00:00:01,2023-11-28 00:00:01,2023-11-27 00:00:01,2023-11-26 00:00:01,改价记录,断货时间,总品类调价记录,每日调价记录,原始行号
0,USBCB72928BLR-C,USBCB72928BLR,83,7.0,87,87.0,89.0,90.0,1,1,...,0.0,1.0,1.0,0.0,2.0,,,,,3
1,USPSP14045D-C,USPSP14045D,17,0.0,18,18.0,18.0,18.0,0,0,...,0.0,0.0,1.0,0.0,0.0,,,,,4
2,USBCB75352ZLR-C,USBCB75352ZLR,4,0.0,4,2.0,0.0,1.0,0,0,...,1.0,1.0,0.0,0.0,3.0,11.27-T10;12.4-H,,,,5
3,USBCB08180ZLR-C,USBCB08180ZLR,62,4.0,62,63.0,63.0,68.0,0,1,...,0.0,0.0,0.0,0.0,0.0,,,,,6
4,USPSP01008D-C,USPSP01008D,19,0.0,19,19.0,19.0,19.0,0,0,...,0.0,0.0,0.0,0.0,0.0,,,,,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26286,USBCB87273LR-C,,0,0.0,0,0.0,0.0,0.0,0,0,...,0.0,0.0,0.0,0.0,,,,,,26289
26287,USCAK22557LR-C,,8,4.0,8,,,,1,0,...,,,,,,,替代料：USCAK09028-C,,,26290
26288,USCAK10009-C,,57,0.0,57,,,,0,0,...,,,,,,,替代料：USCAK10052-C,,,26291
26289,USCA01113-C,,4,0.0,4,,,,0,0,...,,,,,,,替代料: USCA22557-C,,,26292


In [36]:
if timing == 'am':
    df_l = df_last[['SKU', '替代料'] + list(df_last.columns[4:6+1]) + list(df_last.columns[8:14+1]) + [ '改价记录',
                                                                                                         '断货时间',
                                                                                                         '总品类调价记录',
                                                                                                         '每日调价记录',
                                                                                                         '原始行号']].copy()
    df_l.columns = list(df_l.columns[:2]) + ['库存date'] + list(df_l.columns[3:])

else:
    df_l = df_last[['SKU', '替代料'] + list(df_last.columns[5:15+1]) + ['改价记录',
                                                                         '断货时间',
                                                                         '总品类调价记录',
                                                                         '每日调价记录',
                                                                         '原始行号']].copy()

print(list(df_l.columns))
# df_l

['SKU', '替代料', datetime.datetime(2023, 12, 3, 0, 0, 1), datetime.datetime(2023, 12, 1, 0, 0, 1), datetime.datetime(2023, 11, 30, 0, 0, 1), '2023-12-03 00:00:01.1', datetime.datetime(2023, 12, 2, 0, 0, 1), '2023-12-01 00:00:01.1', '2023-11-30 00:00:01.1', datetime.datetime(2023, 11, 29, 0, 0, 1), datetime.datetime(2023, 11, 28, 0, 0, 1), datetime.datetime(2023, 11, 27, 0, 0, 1), datetime.datetime(2023, 11, 26, 0, 0, 1), '改价记录', '断货时间', '总品类调价记录', '每日调价记录', '原始行号']


In [37]:
df_position = df_position[['SKU', 'YKD库存', 'row']]
merge_df_update = df_l
merge_df_update['note'] = ''

In [38]:
# MRP表有的SKU，断货改价表没有的，添加到断货改价表里
list_sku_note = df_mrp[~df_mrp['SKU'].isin(merge_df_update['SKU'])]['SKU'].tolist()
print(len(list_sku_note))
print(list_sku_note)

4
['USCAK09028-C', 'USCAK10052-C', 'USCA22557-C', 'USCA22558-C']


In [39]:
# MRP表有的SKU，断货改价表没有的，添加到断货改价表里
df_last_list = merge_df_update['SKU'].tolist()
length = len(merge_df_update)

for i in range(len(list_sku_note)):
    if list_sku_note[i] in df_last_list:
        merge_df_update.loc[df_last_list.index(list_sku_note[i]), 'note'] = 'last'
    else:
        merge_df_update.loc[length, 'note'] = 'mrp'
        merge_df_update.loc[length, 'SKU'] = list_sku_note[i]
        length += 1
        
merge_df_update = merge_df_update.fillna('')
merge_df_update

,SKU,替代料,2023-12-03 00:00:01,2023-12-01 00:00:01,2023-11-30 00:00:01,2023-12-03 00:00:01.1,2023-12-02 00:00:01,2023-12-01 00:00:01.1,2023-11-30 00:00:01.1,2023-11-29 00:00:01,2023-11-28 00:00:01,2023-11-27 00:00:01,2023-11-26 00:00:01,改价记录,断货时间,总品类调价记录,每日调价记录,原始行号,note
0,USBCB72928BLR-C,USBCB72928BLR,87.0,89.0,90.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,2.0,,,,,3.0,
1,USPSP14045D-C,USPSP14045D,18.0,18.0,18.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,,,,,4.0,
2,USBCB75352ZLR-C,USBCB75352ZLR,2.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,3.0,11.27-T10;12.4-H,,,,5.0,
3,USBCB08180ZLR-C,USBCB08180ZLR,63.0,63.0,68.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,,,,,6.0,
4,USPSP01008D-C,USPSP01008D,19.0,19.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,,7.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26290,USCA01114-C,,,,,0.0,0.0,,,,,,,,替代料: USCA22558-C,,,26293.0,
26291,USCAK09028-C,,,,,,,,,,,,,,,,,,mrp
26292,USCAK10052-C,,,,,,,,,,,,,,,,,,mrp
26293,USCA22557-C,,,,,,,,,,,,,,,,,,mrp


In [40]:
merge_df_update = pd.merge(left=merge_df_update, right=df_position, how='left', left_on='SKU', right_on='SKU', suffixes=('_position','')).fillna('')

print(list(merge_df_update.columns))
merge_df_update

['SKU', '替代料', datetime.datetime(2023, 12, 3, 0, 0, 1), datetime.datetime(2023, 12, 1, 0, 0, 1), datetime.datetime(2023, 11, 30, 0, 0, 1), '2023-12-03 00:00:01.1', datetime.datetime(2023, 12, 2, 0, 0, 1), '2023-12-01 00:00:01.1', '2023-11-30 00:00:01.1', datetime.datetime(2023, 11, 29, 0, 0, 1), datetime.datetime(2023, 11, 28, 0, 0, 1), datetime.datetime(2023, 11, 27, 0, 0, 1), datetime.datetime(2023, 11, 26, 0, 0, 1), '改价记录', '断货时间', '总品类调价记录', '每日调价记录', '原始行号', 'note', 'YKD库存', 'row']


,SKU,替代料,2023-12-03 00:00:01,2023-12-01 00:00:01,2023-11-30 00:00:01,2023-12-03 00:00:01.1,2023-12-02 00:00:01,2023-12-01 00:00:01.1,2023-11-30 00:00:01.1,2023-11-29 00:00:01,...,2023-11-27 00:00:01,2023-11-26 00:00:01,改价记录,断货时间,总品类调价记录,每日调价记录,原始行号,note,YKD库存,row
0,USBCB72928BLR-C,USBCB72928BLR,87.0,89.0,90.0,1.0,1.0,3.0,0.0,1.0,...,0.0,2.0,,,,,3.0,,83.0,15261
1,USPSP14045D-C,USPSP14045D,18.0,18.0,18.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,,,,,4.0,,17.0,14340
2,USBCB75352ZLR-C,USBCB75352ZLR,2.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,3.0,11.27-T10;12.4-H,,,,5.0,,4.0,15283
3,USBCB08180ZLR-C,USBCB08180ZLR,63.0,63.0,68.0,0.0,1.0,4.0,0.0,0.0,...,0.0,0.0,,,,,6.0,,62.0,15264
4,USPSP01008D-C,USPSP01008D,19.0,19.0,19.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,,,,,7.0,,19.0,14713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26290,USCA01114-C,,,,,0.0,0.0,,,,...,,,,替代料: USCA22558-C,,,26293.0,,10.0,32018
26291,USCAK09028-C,,,,,,,,,,...,,,,,,,,mrp,8.0,32015
26292,USCAK10052-C,,,,,,,,,,...,,,,,,,,mrp,57.0,30400
26293,USCA22557-C,,,,,,,,,,...,,,,,,,,mrp,4.0,32017


In [41]:
merge_df_update[merge_df_update.loc[:,'SKU']=='USBCB92640-C']
merge_df_update[merge_df_update.loc[:,'SKU']=='USBCB92640']
print(merge_df_update[merge_df_update.loc[:,'SKU']=='USCAK10052-C'])
merge_df_update[merge_df_update.loc[:,'SKU']=='USCAK10009-C']

                SKU 替代料 2023-12-03 00:00:01 2023-12-01 00:00:01  \
26292  USCAK10052-C                                               

      2023-11-30 00:00:01 2023-12-03 00:00:01.1 2023-12-02 00:00:01  \
26292                                                                 

      2023-12-01 00:00:01.1 2023-11-30 00:00:01.1 2023-11-29 00:00:01  ...  \
26292                                                                  ...   

      2023-11-27 00:00:01 2023-11-26 00:00:01 改价记录 断货时间 总品类调价记录 每日调价记录 原始行号  \
26292                                                                         

      note YKD库存    row  
26292  mrp  57.0  30400  

[1 rows x 21 columns]


,SKU,替代料,2023-12-03 00:00:01,2023-12-01 00:00:01,2023-11-30 00:00:01,2023-12-03 00:00:01.1,2023-12-02 00:00:01,2023-12-01 00:00:01.1,2023-11-30 00:00:01.1,2023-11-29 00:00:01,...,2023-11-27 00:00:01,2023-11-26 00:00:01,改价记录,断货时间,总品类调价记录,每日调价记录,原始行号,note,YKD库存,row
26288,USCAK10009-C,,,,,0.0,0.0,,,,...,,,,替代料：USCAK10052-C,,,26291.0,,57.0,30400


In [42]:
merge_df_update = pd.merge(left=merge_df_update, right=df_mrp, how='left', left_on='SKU', right_on='SKU', suffixes=('','_mrp'))

print(merge_df_update.columns)
# merge_df_update

Index([                  'SKU',                   '替代料',
           2023-12-03 00:00:01,     2023-12-01 00:00:01,
           2023-11-30 00:00:01, '2023-12-03 00:00:01.1',
           2023-12-02 00:00:01, '2023-12-01 00:00:01.1',
       '2023-11-30 00:00:01.1',     2023-11-29 00:00:01,
           2023-11-28 00:00:01,     2023-11-27 00:00:01,
           2023-11-26 00:00:01,                  '改价记录',
                        '断货时间',               '总品类调价记录',
                      '每日调价记录',                  '原始行号',
                        'note',                 'YKD库存',
                         'row',                    '品类',
                          '层级',                    '身份',
                     'row_mrp'],
      dtype='object')


In [43]:
merge_df_update = merge_df_update.merge(df_stock_way, left_on='SKU', right_on='SKU', how='left')

print(list(merge_df_update.columns))
merge_df_update

['SKU', '替代料', datetime.datetime(2023, 12, 3, 0, 0, 1), datetime.datetime(2023, 12, 1, 0, 0, 1), datetime.datetime(2023, 11, 30, 0, 0, 1), '2023-12-03 00:00:01.1', datetime.datetime(2023, 12, 2, 0, 0, 1), '2023-12-01 00:00:01.1', '2023-11-30 00:00:01.1', datetime.datetime(2023, 11, 29, 0, 0, 1), datetime.datetime(2023, 11, 28, 0, 0, 1), datetime.datetime(2023, 11, 27, 0, 0, 1), datetime.datetime(2023, 11, 26, 0, 0, 1), '改价记录', '断货时间', '总品类调价记录', '每日调价记录', '原始行号', 'note', 'YKD库存', 'row', '品类', '层级', '身份', 'row_mrp', '在途总数']


,SKU,替代料,2023-12-03 00:00:01,2023-12-01 00:00:01,2023-11-30 00:00:01,2023-12-03 00:00:01.1,2023-12-02 00:00:01,2023-12-01 00:00:01.1,2023-11-30 00:00:01.1,2023-11-29 00:00:01,...,每日调价记录,原始行号,note,YKD库存,row,品类,层级,身份,row_mrp,在途总数
0,USBCB72928BLR-C,USBCB72928BLR,87.0,89.0,90.0,1.0,1.0,3.0,0.0,1.0,...,,3.0,,83.0,15261,BC,TOP1,老品,15261,61.0
1,USPSP14045D-C,USPSP14045D,18.0,18.0,18.0,0.0,0.0,0.0,0.0,0.0,...,,4.0,,17.0,14340,PSP,TOP10,老品,14340,13.0
2,USBCB75352ZLR-C,USBCB75352ZLR,2.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,,5.0,,4.0,15283,BC,TOP1,断货成长,15283,35.0
3,USBCB08180ZLR-C,USBCB08180ZLR,63.0,63.0,68.0,0.0,1.0,4.0,0.0,0.0,...,,6.0,,62.0,15264,BC,TOP1,老品,15264,30.0
4,USPSP01008D-C,USPSP01008D,19.0,19.0,19.0,0.0,0.0,0.0,0.0,0.0,...,,7.0,,19.0,14713,PSP,普通,老品,14713,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26290,USCA01114-C,,,,,0.0,0.0,,,,...,,26293.0,,10.0,32018,CA,TOP20,老品,32018,25.0
26291,USCAK09028-C,,,,,,,,,,...,,,mrp,8.0,32015,CA,TOP10,老品,32015,0.0
26292,USCAK10052-C,,,,,,,,,,...,,,mrp,57.0,30400,CA,普通,老品,30400,0.0
26293,USCA22557-C,,,,,,,,,,...,,,mrp,4.0,32017,CA,普通,老品,32017,0.0


In [44]:
merge_df_update = pd.merge(left=merge_df_update, right=df_stock, how='left', left_on='SKU',right_on='SKU', suffixes=('', '_stock'))
merge_df_update['YKD库存'] = merge_df_update['YKD库存_stock']
print(merge_df_update.columns)
merge_df_update

Index([                  'SKU',                   '替代料',
           2023-12-03 00:00:01,     2023-12-01 00:00:01,
           2023-11-30 00:00:01, '2023-12-03 00:00:01.1',
           2023-12-02 00:00:01, '2023-12-01 00:00:01.1',
       '2023-11-30 00:00:01.1',     2023-11-29 00:00:01,
           2023-11-28 00:00:01,     2023-11-27 00:00:01,
           2023-11-26 00:00:01,                  '改价记录',
                        '断货时间',               '总品类调价记录',
                      '每日调价记录',                  '原始行号',
                        'note',                 'YKD库存',
                         'row',                    '品类',
                          '层级',                    '身份',
                     'row_mrp',                  '在途总数',
                       '组合料库存',           'YKD库存_stock',
                         '总库存'],
      dtype='object')


,SKU,替代料,2023-12-03 00:00:01,2023-12-01 00:00:01,2023-11-30 00:00:01,2023-12-03 00:00:01.1,2023-12-02 00:00:01,2023-12-01 00:00:01.1,2023-11-30 00:00:01.1,2023-11-29 00:00:01,...,YKD库存,row,品类,层级,身份,row_mrp,在途总数,组合料库存,YKD库存_stock,总库存
0,USBCB72928BLR-C,USBCB72928BLR,87.0,89.0,90.0,1.0,1.0,3.0,0.0,1.0,...,83.0,15261,BC,TOP1,老品,15261,61.0,7.0,83.0,90.0
1,USPSP14045D-C,USPSP14045D,18.0,18.0,18.0,0.0,0.0,0.0,0.0,0.0,...,17.0,14340,PSP,TOP10,老品,14340,13.0,0.0,17.0,17.0
2,USBCB75352ZLR-C,USBCB75352ZLR,2.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,4.0,15283,BC,TOP1,断货成长,15283,35.0,0.0,4.0,4.0
3,USBCB08180ZLR-C,USBCB08180ZLR,63.0,63.0,68.0,0.0,1.0,4.0,0.0,0.0,...,62.0,15264,BC,TOP1,老品,15264,30.0,4.0,62.0,66.0
4,USPSP01008D-C,USPSP01008D,19.0,19.0,19.0,0.0,0.0,0.0,0.0,0.0,...,19.0,14713,PSP,普通,老品,14713,0.0,0.0,19.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26290,USCA01114-C,,,,,0.0,0.0,,,,...,10.0,32018,CA,TOP20,老品,32018,25.0,0.0,10.0,10.0
26291,USCAK09028-C,,,,,,,,,,...,8.0,32015,CA,TOP10,老品,32015,0.0,4.0,8.0,12.0
26292,USCAK10052-C,,,,,,,,,,...,57.0,30400,CA,普通,老品,30400,0.0,0.0,57.0,57.0
26293,USCA22557-C,,,,,,,,,,...,4.0,32017,CA,普通,老品,32017,0.0,0.0,4.0,4.0


In [45]:
# 与替代料匹配更新SKU
merge_df = merge_df_update.merge(df_update, left_on='SKU', right_on='替代料', how='left', suffixes=('','_x')).fillna('')

# 更新SKU
if '替代料_x' in merge_df.columns:
    merge_df['SKU'] = merge_df.apply(lambda row: row['SKU'] if row['替代料_x']=='' else row['主料SKU'], axis=1)
    
    merge_df.drop(columns=['替代料_x', '主料SKU'], inplace=True)
    
print(list(merge_df.columns))
merge_df

['SKU', '替代料', datetime.datetime(2023, 12, 3, 0, 0, 1), datetime.datetime(2023, 12, 1, 0, 0, 1), datetime.datetime(2023, 11, 30, 0, 0, 1), '2023-12-03 00:00:01.1', datetime.datetime(2023, 12, 2, 0, 0, 1), '2023-12-01 00:00:01.1', '2023-11-30 00:00:01.1', datetime.datetime(2023, 11, 29, 0, 0, 1), datetime.datetime(2023, 11, 28, 0, 0, 1), datetime.datetime(2023, 11, 27, 0, 0, 1), datetime.datetime(2023, 11, 26, 0, 0, 1), '改价记录', '断货时间', '总品类调价记录', '每日调价记录', '原始行号', 'note', 'YKD库存', 'row', '品类', '层级', '身份', 'row_mrp', '在途总数', '组合料库存', 'YKD库存_stock', '总库存']


,SKU,替代料,2023-12-03 00:00:01,2023-12-01 00:00:01,2023-11-30 00:00:01,2023-12-03 00:00:01.1,2023-12-02 00:00:01,2023-12-01 00:00:01.1,2023-11-30 00:00:01.1,2023-11-29 00:00:01,...,YKD库存,row,品类,层级,身份,row_mrp,在途总数,组合料库存,YKD库存_stock,总库存
0,USBCB72928BLR-C,USBCB72928BLR,87.0,89.0,90.0,1.0,1.0,3.0,0.0,1.0,...,83.0,15261,BC,TOP1,老品,15261,61.0,7.0,83.0,90.0
1,USPSP14045D-C,USPSP14045D,18.0,18.0,18.0,0.0,0.0,0.0,0.0,0.0,...,17.0,14340,PSP,TOP10,老品,14340,13.0,0.0,17.0,17.0
2,USBCB75352ZLR-C,USBCB75352ZLR,2.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,4.0,15283,BC,TOP1,断货成长,15283,35.0,0.0,4.0,4.0
3,USBCB08180ZLR-C,USBCB08180ZLR,63.0,63.0,68.0,0.0,1.0,4.0,0.0,0.0,...,62.0,15264,BC,TOP1,老品,15264,30.0,4.0,62.0,66.0
4,USPSP01008D-C,USPSP01008D,19.0,19.0,19.0,0.0,0.0,0.0,0.0,0.0,...,19.0,14713,PSP,普通,老品,14713,0.0,0.0,19.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26290,USCA01114-C,,,,,0.0,0.0,,,,...,10.0,32018,CA,TOP20,老品,32018,25.0,0.0,10.0,10.0
26291,USCAK22557LR-C,,,,,,,,,,...,8.0,32015,CA,TOP10,老品,32015,0.0,4.0,8.0,12.0
26292,USCAK10009-C,,,,,,,,,,...,57.0,30400,CA,普通,老品,30400,0.0,0.0,57.0,57.0
26293,USCA01113-C,,,,,,,,,,...,4.0,32017,CA,普通,老品,32017,0.0,0.0,4.0,4.0


In [46]:
merge_df,l = DEduplicate(merge_df)

In [47]:
merge_df[merge_df.loc[:,'SKU']=='USCAK10052-C']['note']
# merge_df[merge_df.loc[:,'SKU']=='USCAK10009-C']

Series([], Name: note, dtype: object)

In [48]:
df = merge_df
df['改价'] = ''
df['weekly_sale'] = ''
df['改价+断货'] = ''
df['总品类调价记录_2'] = ''
df['合计改价记录'] = ''

if timing == 'am':
    df = df[['层级',
             'SKU',
             '替代料',
             '品类',
             '身份',
             '在途总数',
             '改价',
             '组合料库存',
             'YKD库存'] + list(df.columns)[2:11+1] + ['weekly_sale',
                                                       '改价记录', 
                                                       '断货时间', 
                                                       '改价+断货',
                                                       '总库存',
                                                       '总品类调价记录',
                                                       '每日调价记录',
                                                       '总品类调价记录_2',
                                                       '合计改价记录',
                                                       'note',
                                                       '原始行号',
                                                       'row']].copy()
else:
    df = df[['层级',
             'SKU',
             '替代料',
             '品类',
             '身份',
             '在途总数',
             '改价',
             '组合料库存',
             'YKD库存'] + list(df.columns)[2:12+1] + ['weekly_sale',
                                                       '改价记录', 
                                                       '断货时间', 
                                                       '改价+断货',
                                                       '总库存',
                                                       '总品类调价记录',
                                                       '每日调价记录',
                                                       '总品类调价记录_2',
                                                       '合计改价记录',
                                                       'note',
                                                       '原始行号',
                                                       'row']].copy()
print(list(df.columns))
df

['层级', 'SKU', '替代料', '品类', '身份', '在途总数', '改价', '组合料库存', 'YKD库存', datetime.datetime(2023, 12, 3, 0, 0, 1), datetime.datetime(2023, 12, 1, 0, 0, 1), datetime.datetime(2023, 11, 30, 0, 0, 1), '2023-12-03 00:00:01.1', datetime.datetime(2023, 12, 2, 0, 0, 1), '2023-12-01 00:00:01.1', '2023-11-30 00:00:01.1', datetime.datetime(2023, 11, 29, 0, 0, 1), datetime.datetime(2023, 11, 28, 0, 0, 1), datetime.datetime(2023, 11, 27, 0, 0, 1), datetime.datetime(2023, 11, 26, 0, 0, 1), 'weekly_sale', '改价记录', '断货时间', '改价+断货', '总库存', '总品类调价记录', '每日调价记录', '总品类调价记录_2', '合计改价记录', 'note', '原始行号', 'row']


,层级,SKU,替代料,品类,身份,在途总数,改价,组合料库存,YKD库存,2023-12-03 00:00:01,...,断货时间,改价+断货,总库存,总品类调价记录,每日调价记录,总品类调价记录_2,合计改价记录,note,原始行号,row
0,TOP1,USBCB72928BLR-C,USBCB72928BLR,BC,老品,61.0,,7.0,83.0,87.0,...,,,90.0,,,,,,3.0,15261
1,TOP10,USPSP14045D-C,USPSP14045D,PSP,老品,13.0,,0.0,17.0,18.0,...,,,17.0,,,,,,4.0,14340
2,TOP1,USBCB75352ZLR-C,USBCB75352ZLR,BC,断货成长,35.0,,0.0,4.0,2.0,...,,,4.0,,,,,,5.0,15283
3,TOP1,USBCB08180ZLR-C,USBCB08180ZLR,BC,老品,30.0,,4.0,62.0,63.0,...,,,66.0,,,,,,6.0,15264
4,普通,USPSP01008D-C,USPSP01008D,PSP,老品,0.0,,0.0,19.0,19.0,...,,,19.0,,,,,,7.0,14713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26286,普通,USBCB87273LR-C,,BC,新品,6.0,,0.0,0.0,0.0,...,,,0.0,,,,,,26289.0,
26287,TOP10,USCAK22557LR-C,,CA,老品,0.0,,4.0,8.0,,...,,,12.0,,,,,mrp,,32015
26288,普通,USCAK10009-C,,CA,老品,0.0,,0.0,57.0,,...,,,57.0,,,,,mrp,,30400
26289,普通,USCA01113-C,,CA,老品,0.0,,0.0,4.0,,...,,,4.0,,,,,mrp,,32017


# 6.其他信息

In [49]:
df_missing = pd.read_excel(stcok,
                           sheet_name=1,
                           names=['SKU', '无Record ID'],
                           usecols='D,F')

df_missing = df_missing.groupby('SKU', as_index=False).sum()

df_missing

,SKU,无Record ID
0,USABS13304-C,1
1,USABS5970-C,1
2,USACH14045-C,1
3,USBCB84728B-C,1
4,USBCB91011LR-C,2
...,...,...
154,USWR41002,1
155,USWR49509-C,1
156,USWR52312HY-C,1
157,USWRM41102-C,1


In [50]:
df = pd.merge(left=df, right=df_missing, how='left', left_on='SKU', right_on='SKU').fillna(0)

print(list(df.columns))
df

['层级', 'SKU', '替代料', '品类', '身份', '在途总数', '改价', '组合料库存', 'YKD库存', datetime.datetime(2023, 12, 3, 0, 0, 1), datetime.datetime(2023, 12, 1, 0, 0, 1), datetime.datetime(2023, 11, 30, 0, 0, 1), '2023-12-03 00:00:01.1', datetime.datetime(2023, 12, 2, 0, 0, 1), '2023-12-01 00:00:01.1', '2023-11-30 00:00:01.1', datetime.datetime(2023, 11, 29, 0, 0, 1), datetime.datetime(2023, 11, 28, 0, 0, 1), datetime.datetime(2023, 11, 27, 0, 0, 1), datetime.datetime(2023, 11, 26, 0, 0, 1), 'weekly_sale', '改价记录', '断货时间', '改价+断货', '总库存', '总品类调价记录', '每日调价记录', '总品类调价记录_2', '合计改价记录', 'note', '原始行号', 'row', '无Record ID']


,层级,SKU,替代料,品类,身份,在途总数,改价,组合料库存,YKD库存,2023-12-03 00:00:01,...,改价+断货,总库存,总品类调价记录,每日调价记录,总品类调价记录_2,合计改价记录,note,原始行号,row,无Record ID
0,TOP1,USBCB72928BLR-C,USBCB72928BLR,BC,老品,61.0,,7.0,83.0,87.0,...,,90.0,,,,,,3.0,15261,0.0
1,TOP10,USPSP14045D-C,USPSP14045D,PSP,老品,13.0,,0.0,17.0,18.0,...,,17.0,,,,,,4.0,14340,0.0
2,TOP1,USBCB75352ZLR-C,USBCB75352ZLR,BC,断货成长,35.0,,0.0,4.0,2.0,...,,4.0,,,,,,5.0,15283,0.0
3,TOP1,USBCB08180ZLR-C,USBCB08180ZLR,BC,老品,30.0,,4.0,62.0,63.0,...,,66.0,,,,,,6.0,15264,0.0
4,普通,USPSP01008D-C,USPSP01008D,PSP,老品,0.0,,0.0,19.0,19.0,...,,19.0,,,,,,7.0,14713,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26286,普通,USBCB87273LR-C,,BC,新品,6.0,,0.0,0.0,0.0,...,,0.0,,,,,,26289.0,,0.0
26287,TOP10,USCAK22557LR-C,,CA,老品,0.0,,4.0,8.0,,...,,12.0,,,,,mrp,,32015,0.0
26288,普通,USCAK10009-C,,CA,老品,0.0,,0.0,57.0,,...,,57.0,,,,,mrp,,30400,0.0
26289,普通,USCA01113-C,,CA,老品,0.0,,0.0,4.0,,...,,4.0,,,,,mrp,,32017,0.0


In [51]:
df_missing = pd.merge(left=df_missing, right=df_position, how='left', left_on='SKU', right_on='SKU').fillna('')
df_missing = df_missing[df_missing.loc[:,'row'] != ''][['SKU', '无Record ID']].copy()
df_missing = df_missing.groupby('SKU', as_index=False).sum()
df_missing

,SKU,无Record ID
0,USABS13304-C,1
1,USABS5970-C,1
2,USACH14045-C,1
3,USBCB84728B-C,1
4,USBCB91011LR-C,2
...,...,...
133,USWR26992M-C,1
134,USWR49509-C,1
135,USWR52312HY-C,1
136,USWRM41102-C,1


In [52]:
df = pd.merge(left=df, right=df_missing, how='left', left_on='SKU', right_on='SKU', suffixes=('_missing1', '_missing2')).fillna('')

print(df.columns)
df

Index([                   '层级',                   'SKU',
                         '替代料',                    '品类',
                          '身份',                  '在途总数',
                          '改价',                 '组合料库存',
                       'YKD库存',     2023-12-03 00:00:01,
           2023-12-01 00:00:01,     2023-11-30 00:00:01,
       '2023-12-03 00:00:01.1',     2023-12-02 00:00:01,
       '2023-12-01 00:00:01.1', '2023-11-30 00:00:01.1',
           2023-11-29 00:00:01,     2023-11-28 00:00:01,
           2023-11-27 00:00:01,     2023-11-26 00:00:01,
                 'weekly_sale',                  '改价记录',
                        '断货时间',                 '改价+断货',
                         '总库存',               '总品类调价记录',
                      '每日调价记录',             '总品类调价记录_2',
                      '合计改价记录',                  'note',
                        '原始行号',                   'row',
         '无Record ID_missing1',   '无Record ID_missing2'],
      dtype='object')


,层级,SKU,替代料,品类,身份,在途总数,改价,组合料库存,YKD库存,2023-12-03 00:00:01,...,总库存,总品类调价记录,每日调价记录,总品类调价记录_2,合计改价记录,note,原始行号,row,无Record ID_missing1,无Record ID_missing2
0,TOP1,USBCB72928BLR-C,USBCB72928BLR,BC,老品,61.0,,7.0,83.0,87.0,...,90.0,,,,,,3.0,15261,0.0,
1,TOP10,USPSP14045D-C,USPSP14045D,PSP,老品,13.0,,0.0,17.0,18.0,...,17.0,,,,,,4.0,14340,0.0,
2,TOP1,USBCB75352ZLR-C,USBCB75352ZLR,BC,断货成长,35.0,,0.0,4.0,2.0,...,4.0,,,,,,5.0,15283,0.0,
3,TOP1,USBCB08180ZLR-C,USBCB08180ZLR,BC,老品,30.0,,4.0,62.0,63.0,...,66.0,,,,,,6.0,15264,0.0,
4,普通,USPSP01008D-C,USPSP01008D,PSP,老品,0.0,,0.0,19.0,19.0,...,19.0,,,,,,7.0,14713,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26286,普通,USBCB87273LR-C,,BC,新品,6.0,,0.0,0.0,0.0,...,0.0,,,,,,26289.0,,0.0,
26287,TOP10,USCAK22557LR-C,,CA,老品,0.0,,4.0,8.0,,...,12.0,,,,,mrp,,32015,0.0,
26288,普通,USCAK10009-C,,CA,老品,0.0,,0.0,57.0,,...,57.0,,,,,mrp,,30400,0.0,
26289,普通,USCA01113-C,,CA,老品,0.0,,0.0,4.0,,...,4.0,,,,,mrp,,32017,0.0,


In [53]:
df.columns

Index([                   '层级',                   'SKU',
                         '替代料',                    '品类',
                          '身份',                  '在途总数',
                          '改价',                 '组合料库存',
                       'YKD库存',     2023-12-03 00:00:01,
           2023-12-01 00:00:01,     2023-11-30 00:00:01,
       '2023-12-03 00:00:01.1',     2023-12-02 00:00:01,
       '2023-12-01 00:00:01.1', '2023-11-30 00:00:01.1',
           2023-11-29 00:00:01,     2023-11-28 00:00:01,
           2023-11-27 00:00:01,     2023-11-26 00:00:01,
                 'weekly_sale',                  '改价记录',
                        '断货时间',                 '改价+断货',
                         '总库存',               '总品类调价记录',
                      '每日调价记录',             '总品类调价记录_2',
                      '合计改价记录',                  'note',
                        '原始行号',                   'row',
         '无Record ID_missing1',   '无Record ID_missing2'],
      dtype='object')

In [54]:
df['无Record ID'] = [df.loc[i,'无Record ID_missing2'] if df.loc[i,'无Record ID_missing2'] != '' else df.loc[i,'无Record ID_missing1'] for i in range(len(df))]
df['无Record ID'] = pd.to_numeric(df['无Record ID'], errors='coerce')
df[ 'YKD库存'] = pd.to_numeric(df['YKD库存'], errors='coerce')
df['总库存'] = pd.to_numeric(df['总库存'], errors='coerce')
df['实时库存'] = [df.loc[i, 'YKD库存'] - df.loc[i, '无Record ID'] if df.loc[i, 'note'] != 'last' else '' for i in range(len(df))]
df.loc[:,'总库存'] = df.loc[:, '总库存'] - df.loc[:, '无Record ID']
print(list(df.columns))
df

['层级', 'SKU', '替代料', '品类', '身份', '在途总数', '改价', '组合料库存', 'YKD库存', datetime.datetime(2023, 12, 3, 0, 0, 1), datetime.datetime(2023, 12, 1, 0, 0, 1), datetime.datetime(2023, 11, 30, 0, 0, 1), '2023-12-03 00:00:01.1', datetime.datetime(2023, 12, 2, 0, 0, 1), '2023-12-01 00:00:01.1', '2023-11-30 00:00:01.1', datetime.datetime(2023, 11, 29, 0, 0, 1), datetime.datetime(2023, 11, 28, 0, 0, 1), datetime.datetime(2023, 11, 27, 0, 0, 1), datetime.datetime(2023, 11, 26, 0, 0, 1), 'weekly_sale', '改价记录', '断货时间', '改价+断货', '总库存', '总品类调价记录', '每日调价记录', '总品类调价记录_2', '合计改价记录', 'note', '原始行号', 'row', '无Record ID_missing1', '无Record ID_missing2', '无Record ID', '实时库存']


,层级,SKU,替代料,品类,身份,在途总数,改价,组合料库存,YKD库存,2023-12-03 00:00:01,...,每日调价记录,总品类调价记录_2,合计改价记录,note,原始行号,row,无Record ID_missing1,无Record ID_missing2,无Record ID,实时库存
0,TOP1,USBCB72928BLR-C,USBCB72928BLR,BC,老品,61.0,,7.0,83.0,87.0,...,,,,,3.0,15261,0.0,,0.0,83.0
1,TOP10,USPSP14045D-C,USPSP14045D,PSP,老品,13.0,,0.0,17.0,18.0,...,,,,,4.0,14340,0.0,,0.0,17.0
2,TOP1,USBCB75352ZLR-C,USBCB75352ZLR,BC,断货成长,35.0,,0.0,4.0,2.0,...,,,,,5.0,15283,0.0,,0.0,4.0
3,TOP1,USBCB08180ZLR-C,USBCB08180ZLR,BC,老品,30.0,,4.0,62.0,63.0,...,,,,,6.0,15264,0.0,,0.0,62.0
4,普通,USPSP01008D-C,USPSP01008D,PSP,老品,0.0,,0.0,19.0,19.0,...,,,,,7.0,14713,0.0,,0.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26286,普通,USBCB87273LR-C,,BC,新品,6.0,,0.0,0.0,0.0,...,,,,,26289.0,,0.0,,0.0,0.0
26287,TOP10,USCAK22557LR-C,,CA,老品,0.0,,4.0,8.0,,...,,,,mrp,,32015,0.0,,0.0,8.0
26288,普通,USCAK10009-C,,CA,老品,0.0,,0.0,57.0,,...,,,,mrp,,30400,0.0,,0.0,57.0
26289,普通,USCA01113-C,,CA,老品,0.0,,0.0,4.0,,...,,,,mrp,,32017,0.0,,0.0,4.0


In [55]:
df_tmp = df
if timing == 'am':
    df_tmp = df_tmp[['层级',
             'SKU',
             '替代料',
             '品类',
             '身份',
             '在途总数',
             '改价',
             '无Record ID',
             '实时库存',
             '组合料库存',
             'YKD库存'] + list(df_tmp.columns)[9:18+1] + ['weekly_sale',
                                                       '改价记录', 
                                                       '断货时间', 
                                                       '改价+断货',
                                                       '总库存',
                                                       '总品类调价记录',
                                                       '每日调价记录',
                                                       '总品类调价记录_2',
                                                       '合计改价记录',
                                                       'note',
                                                       '原始行号',
                                                       'row']].copy()
else:
    df_tmp = df_tmp[['层级',
             'SKU',
             '替代料',
             '品类',
             '身份',
             '在途总数',
             '改价',
             '无Record ID',
             '实时库存',
             '组合料库存',
             'YKD库存'] + list(df_tmp.columns)[9:19+1] + ['weekly_sale',
                                                       '改价记录', 
                                                       '断货时间', 
                                                       '改价+断货',
                                                       '总库存',
                                                       '总品类调价记录',
                                                       '每日调价记录',
                                                       '总品类调价记录_2',
                                                       '合计改价记录',
                                                       'note',
                                                       '原始行号',
                                                       'row']].copy()
print(list(df_tmp.columns))
df = df_tmp
df

['层级', 'SKU', '替代料', '品类', '身份', '在途总数', '改价', '无Record ID', '实时库存', '组合料库存', 'YKD库存', datetime.datetime(2023, 12, 3, 0, 0, 1), datetime.datetime(2023, 12, 1, 0, 0, 1), datetime.datetime(2023, 11, 30, 0, 0, 1), '2023-12-03 00:00:01.1', datetime.datetime(2023, 12, 2, 0, 0, 1), '2023-12-01 00:00:01.1', '2023-11-30 00:00:01.1', datetime.datetime(2023, 11, 29, 0, 0, 1), datetime.datetime(2023, 11, 28, 0, 0, 1), datetime.datetime(2023, 11, 27, 0, 0, 1), datetime.datetime(2023, 11, 26, 0, 0, 1), 'weekly_sale', '改价记录', '断货时间', '改价+断货', '总库存', '总品类调价记录', '每日调价记录', '总品类调价记录_2', '合计改价记录', 'note', '原始行号', 'row']


,层级,SKU,替代料,品类,身份,在途总数,改价,无Record ID,实时库存,组合料库存,...,断货时间,改价+断货,总库存,总品类调价记录,每日调价记录,总品类调价记录_2,合计改价记录,note,原始行号,row
0,TOP1,USBCB72928BLR-C,USBCB72928BLR,BC,老品,61.0,,0.0,83.0,7.0,...,,,90.0,,,,,,3.0,15261
1,TOP10,USPSP14045D-C,USPSP14045D,PSP,老品,13.0,,0.0,17.0,0.0,...,,,17.0,,,,,,4.0,14340
2,TOP1,USBCB75352ZLR-C,USBCB75352ZLR,BC,断货成长,35.0,,0.0,4.0,0.0,...,,,4.0,,,,,,5.0,15283
3,TOP1,USBCB08180ZLR-C,USBCB08180ZLR,BC,老品,30.0,,0.0,62.0,4.0,...,,,66.0,,,,,,6.0,15264
4,普通,USPSP01008D-C,USPSP01008D,PSP,老品,0.0,,0.0,19.0,0.0,...,,,19.0,,,,,,7.0,14713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26286,普通,USBCB87273LR-C,,BC,新品,6.0,,0.0,0.0,0.0,...,,,0.0,,,,,,26289.0,
26287,TOP10,USCAK22557LR-C,,CA,老品,0.0,,0.0,8.0,4.0,...,,,12.0,,,,,mrp,,32015
26288,普通,USCAK10009-C,,CA,老品,0.0,,0.0,57.0,0.0,...,,,57.0,,,,,mrp,,30400
26289,普通,USCA01113-C,,CA,老品,0.0,,0.0,4.0,0.0,...,,,4.0,,,,,mrp,,32017


In [56]:
if timing == 'am':
    df_sales = pd.read_excel(stcok,
                             sheet_name=2,
                             usecols='D,F')
    
    df_sales = df_sales.groupby('SKU', as_index=False).sum()

In [57]:
if timing == 'am':
    df = pd.merge(left=df, right=df_sales, how='left', left_on='SKU', right_on='SKU', suffixes=('','_sales')).fillna(0)

In [58]:
if timing == 'am':
    df_sales = pd.merge(left=df_sales, right=df_position, how='left', left_on='SKU', right_on='SKU', suffixes=('_x','')).fillna('')
    df_sales = df_sales[df_sales['row'] != ''][['购买数量', 'row']].copy()
    df_sales = df_sales.groupby('row', as_index=False).sum()
    df_sales

In [59]:
if timing == 'am':
#     df_tmp = df
    df = pd.merge(left=df, right=df_sales, how='left', left_on='row', right_on='row', suffixes=('_sales1', '_sales2')).fillna('')
    df['销量date'] = [df.loc[i, '购买数量_sales2'] if df.loc[i, '购买数量_sales2'] != '' else df.loc[i, '购买数量_sales1']  for i in range(len(df))]
    print(list(df.columns))
    
    df

In [60]:
if timing == 'am':
    df = df[['层级',
             'SKU',
             '替代料',
             '品类',
             '身份',
             '在途总数',
             '改价',
             '无Record ID',
             '实时库存',
             '组合料库存',
             'YKD库存'] + list(df.columns)[11:13+1]+ ['销量date'] + list(df.columns)[14:20+1] + ['weekly_sale',
                                                                                                 '改价记录', 
                                                                                                 '断货时间', 
                                                                                                 '改价+断货',
                                                                                                 '总库存',
                                                                                                 '总品类调价记录',
                                                                                                 '每日调价记录',
                                                                                                 '总品类调价记录_2',
                                                                                                 '合计改价记录',
                                                                                                 'note',
                                                                                                 '原始行号']].copy()

print(list(df.columns))
df

['层级', 'SKU', '替代料', '品类', '身份', '在途总数', '改价', '无Record ID', '实时库存', '组合料库存', 'YKD库存', datetime.datetime(2023, 12, 3, 0, 0, 1), datetime.datetime(2023, 12, 1, 0, 0, 1), datetime.datetime(2023, 11, 30, 0, 0, 1), '2023-12-03 00:00:01.1', datetime.datetime(2023, 12, 2, 0, 0, 1), '2023-12-01 00:00:01.1', '2023-11-30 00:00:01.1', datetime.datetime(2023, 11, 29, 0, 0, 1), datetime.datetime(2023, 11, 28, 0, 0, 1), datetime.datetime(2023, 11, 27, 0, 0, 1), datetime.datetime(2023, 11, 26, 0, 0, 1), 'weekly_sale', '改价记录', '断货时间', '改价+断货', '总库存', '总品类调价记录', '每日调价记录', '总品类调价记录_2', '合计改价记录', 'note', '原始行号', 'row']


,层级,SKU,替代料,品类,身份,在途总数,改价,无Record ID,实时库存,组合料库存,...,断货时间,改价+断货,总库存,总品类调价记录,每日调价记录,总品类调价记录_2,合计改价记录,note,原始行号,row
0,TOP1,USBCB72928BLR-C,USBCB72928BLR,BC,老品,61.0,,0.0,83.0,7.0,...,,,90.0,,,,,,3.0,15261
1,TOP10,USPSP14045D-C,USPSP14045D,PSP,老品,13.0,,0.0,17.0,0.0,...,,,17.0,,,,,,4.0,14340
2,TOP1,USBCB75352ZLR-C,USBCB75352ZLR,BC,断货成长,35.0,,0.0,4.0,0.0,...,,,4.0,,,,,,5.0,15283
3,TOP1,USBCB08180ZLR-C,USBCB08180ZLR,BC,老品,30.0,,0.0,62.0,4.0,...,,,66.0,,,,,,6.0,15264
4,普通,USPSP01008D-C,USPSP01008D,PSP,老品,0.0,,0.0,19.0,0.0,...,,,19.0,,,,,,7.0,14713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26286,普通,USBCB87273LR-C,,BC,新品,6.0,,0.0,0.0,0.0,...,,,0.0,,,,,,26289.0,
26287,TOP10,USCAK22557LR-C,,CA,老品,0.0,,0.0,8.0,4.0,...,,,12.0,,,,,mrp,,32015
26288,普通,USCAK10009-C,,CA,老品,0.0,,0.0,57.0,0.0,...,,,57.0,,,,,mrp,,30400
26289,普通,USCA01113-C,,CA,老品,0.0,,0.0,4.0,0.0,...,,,4.0,,,,,mrp,,32017


In [61]:
offset = 3

df['改价'] = ['=IFERROR(IF(I' + str(i+3) + '-$L' + str(i+3) + '>0,"海运上架",IF(I'+ str(i+3) + '<=0,"断货","")),"")' for i in range(len(df))]
df['weekly_sale'] = ['=SUM(O' + str(i+3) + ':V' + str(i+3) +')' for i in range(len(df))]
df['改价+断货'] = [ '=TEXTJOIN(";",1,X' + str(i+3) + ',Y' + str(i+3) + ')' for i in range(len(df))]
df['总品类调价记录_2'] = ['=TEXTJOIN(";",TRUE,AB' + str(i+3) + ':AC' + str(i+3) + ')' for i in range(len(df))]
df['合计改价记录'] = ['=TEXTJOIN(";",1,AD' + str(i+3) + ',X' + str(i+3) + ')' for i in range(len(df))]

In [62]:
print(df.columns)

Index([                   '层级',                   'SKU',
                         '替代料',                    '品类',
                          '身份',                  '在途总数',
                          '改价',            '无Record ID',
                        '实时库存',                 '组合料库存',
                       'YKD库存',     2023-12-03 00:00:01,
           2023-12-01 00:00:01,     2023-11-30 00:00:01,
       '2023-12-03 00:00:01.1',     2023-12-02 00:00:01,
       '2023-12-01 00:00:01.1', '2023-11-30 00:00:01.1',
           2023-11-29 00:00:01,     2023-11-28 00:00:01,
           2023-11-27 00:00:01,     2023-11-26 00:00:01,
                 'weekly_sale',                  '改价记录',
                        '断货时间',                 '改价+断货',
                         '总库存',               '总品类调价记录',
                      '每日调价记录',             '总品类调价记录_2',
                      '合计改价记录',                  'note',
                        '原始行号',                   'row'],
      dtype='object')


In [63]:
df_tmp = df
df_tmp = pd.merge(left=df_tmp, right=df_day_category, how='left', left_on='SKU', right_on='SKU', suffixes=('','_last_record')).fillna('')
df_tmp['每日调价记录'] = df_tmp['调价记录']
print(df_tmp.columns)
# df_tmp

Index([                   '层级',                   'SKU',
                         '替代料',                    '品类',
                          '身份',                  '在途总数',
                          '改价',            '无Record ID',
                        '实时库存',                 '组合料库存',
                       'YKD库存',     2023-12-03 00:00:01,
           2023-12-01 00:00:01,     2023-11-30 00:00:01,
       '2023-12-03 00:00:01.1',     2023-12-02 00:00:01,
       '2023-12-01 00:00:01.1', '2023-11-30 00:00:01.1',
           2023-11-29 00:00:01,     2023-11-28 00:00:01,
           2023-11-27 00:00:01,     2023-11-26 00:00:01,
                 'weekly_sale',                  '改价记录',
                        '断货时间',                 '改价+断货',
                         '总库存',               '总品类调价记录',
                      '每日调价记录',             '总品类调价记录_2',
                      '合计改价记录',                  'note',
                        '原始行号',                   'row',
                        '调价记录']

In [64]:
# df = df_tmp

if timing == 'am':
    df_tmp = df_tmp[['层级',
                     'SKU',
                     '替代料',
                     '品类',
                     '身份',
                     '在途总数',
                     '改价',
                     '无Record ID',
                     '实时库存',
                     '组合料库存',
                     'YKD库存'] + list(df_tmp.columns)[11:21+1] + ['weekly_sale',
                                                                     '改价记录', 
                                                                     '断货时间', 
                                                                     '改价+断货',
                                                                     '总库存',
                                                                     '总品类调价记录',
                                                                     '每日调价记录',
                                                                     '总品类调价记录_2',
                                                                     '合计改价记录',
                                                                     'note',
                                                                     '原始行号']].copy()

    print(list(df_tmp.columns))
    df_tmp
else:
    df_tmp = df_tmp[['层级',
                     'SKU',
                     '替代料',
                     '品类',
                     '身份',
                     '在途总数',
                     '改价',
                     '无Record ID',
                     '实时库存',
                     '组合料库存',
                     'YKD库存'] + list(df_tmp.columns)[11:21+1] + ['weekly_sale',
                                                                     '改价记录', 
                                                                     '断货时间', 
                                                                     '改价+断货',
                                                                     '总库存',
                                                                     '总品类调价记录',
                                                                     '每日调价记录',
                                                                     '总品类调价记录_2',
                                                                     '合计改价记录',
                                                                     'note',
                                                                     '原始行号']].copy()
# df = df_tmp
print(df_tmp.columns)
df_tmp

Index([                   '层级',                   'SKU',
                         '替代料',                    '品类',
                          '身份',                  '在途总数',
                          '改价',            '无Record ID',
                        '实时库存',                 '组合料库存',
                       'YKD库存',     2023-12-03 00:00:01,
           2023-12-01 00:00:01,     2023-11-30 00:00:01,
       '2023-12-03 00:00:01.1',     2023-12-02 00:00:01,
       '2023-12-01 00:00:01.1', '2023-11-30 00:00:01.1',
           2023-11-29 00:00:01,     2023-11-28 00:00:01,
           2023-11-27 00:00:01,     2023-11-26 00:00:01,
                 'weekly_sale',                  '改价记录',
                        '断货时间',                 '改价+断货',
                         '总库存',               '总品类调价记录',
                      '每日调价记录',             '总品类调价记录_2',
                      '合计改价记录',                  'note',
                        '原始行号'],
      dtype='object')


,层级,SKU,替代料,品类,身份,在途总数,改价,无Record ID,实时库存,组合料库存,...,改价记录,断货时间,改价+断货,总库存,总品类调价记录,每日调价记录,总品类调价记录_2,合计改价记录,note,原始行号
0,TOP1,USBCB72928BLR-C,USBCB72928BLR,BC,老品,61.0,"=IFERROR(IF(I3-$L3>0,""海运上架"",IF(I3<=0,""断货"",""""))...",0.0,83.0,7.0,...,,,"=TEXTJOIN("";"",1,X3,Y3)",90.0,,,"=TEXTJOIN("";"",TRUE,AB3:AC3)","=TEXTJOIN("";"",1,AD3,X3)",,3.0
1,TOP10,USPSP14045D-C,USPSP14045D,PSP,老品,13.0,"=IFERROR(IF(I4-$L4>0,""海运上架"",IF(I4<=0,""断货"",""""))...",0.0,17.0,0.0,...,,,"=TEXTJOIN("";"",1,X4,Y4)",17.0,,,"=TEXTJOIN("";"",TRUE,AB4:AC4)","=TEXTJOIN("";"",1,AD4,X4)",,4.0
2,TOP1,USBCB75352ZLR-C,USBCB75352ZLR,BC,断货成长,35.0,"=IFERROR(IF(I5-$L5>0,""海运上架"",IF(I5<=0,""断货"",""""))...",0.0,4.0,0.0,...,11.27-T10;12.4-H,,"=TEXTJOIN("";"",1,X5,Y5)",4.0,,,"=TEXTJOIN("";"",TRUE,AB5:AC5)","=TEXTJOIN("";"",1,AD5,X5)",,5.0
3,TOP1,USBCB08180ZLR-C,USBCB08180ZLR,BC,老品,30.0,"=IFERROR(IF(I6-$L6>0,""海运上架"",IF(I6<=0,""断货"",""""))...",0.0,62.0,4.0,...,,,"=TEXTJOIN("";"",1,X6,Y6)",66.0,,,"=TEXTJOIN("";"",TRUE,AB6:AC6)","=TEXTJOIN("";"",1,AD6,X6)",,6.0
4,普通,USPSP01008D-C,USPSP01008D,PSP,老品,0.0,"=IFERROR(IF(I7-$L7>0,""海运上架"",IF(I7<=0,""断货"",""""))...",0.0,19.0,0.0,...,,,"=TEXTJOIN("";"",1,X7,Y7)",19.0,,,"=TEXTJOIN("";"",TRUE,AB7:AC7)","=TEXTJOIN("";"",1,AD7,X7)",,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26286,普通,USBCB87273LR-C,,BC,新品,6.0,"=IFERROR(IF(I26289-$L26289>0,""海运上架"",IF(I26289<...",0.0,0.0,0.0,...,,,"=TEXTJOIN("";"",1,X26289,Y26289)",0.0,,,"=TEXTJOIN("";"",TRUE,AB26289:AC26289)","=TEXTJOIN("";"",1,AD26289,X26289)",,26289.0
26287,TOP10,USCAK22557LR-C,,CA,老品,0.0,"=IFERROR(IF(I26290-$L26290>0,""海运上架"",IF(I26290<...",0.0,8.0,4.0,...,,,"=TEXTJOIN("";"",1,X26290,Y26290)",12.0,,,"=TEXTJOIN("";"",TRUE,AB26290:AC26290)","=TEXTJOIN("";"",1,AD26290,X26290)",mrp,
26288,普通,USCAK10009-C,,CA,老品,0.0,"=IFERROR(IF(I26291-$L26291>0,""海运上架"",IF(I26291<...",0.0,57.0,0.0,...,,,"=TEXTJOIN("";"",1,X26291,Y26291)",57.0,,,"=TEXTJOIN("";"",TRUE,AB26291:AC26291)","=TEXTJOIN("";"",1,AD26291,X26291)",mrp,
26289,普通,USCA01113-C,,CA,老品,0.0,"=IFERROR(IF(I26292-$L26292>0,""海运上架"",IF(I26292<...",0.0,4.0,0.0,...,,,"=TEXTJOIN("";"",1,X26292,Y26292)",4.0,,,"=TEXTJOIN("";"",TRUE,AB26292:AC26292)","=TEXTJOIN("";"",1,AD26292,X26292)",mrp,


In [65]:
df_tmp.columns

Index([                   '层级',                   'SKU',
                         '替代料',                    '品类',
                          '身份',                  '在途总数',
                          '改价',            '无Record ID',
                        '实时库存',                 '组合料库存',
                       'YKD库存',     2023-12-03 00:00:01,
           2023-12-01 00:00:01,     2023-11-30 00:00:01,
       '2023-12-03 00:00:01.1',     2023-12-02 00:00:01,
       '2023-12-01 00:00:01.1', '2023-11-30 00:00:01.1',
           2023-11-29 00:00:01,     2023-11-28 00:00:01,
           2023-11-27 00:00:01,     2023-11-26 00:00:01,
                 'weekly_sale',                  '改价记录',
                        '断货时间',                 '改价+断货',
                         '总库存',               '总品类调价记录',
                      '每日调价记录',             '总品类调价记录_2',
                      '合计改价记录',                  'note',
                        '原始行号'],
      dtype='object')

In [66]:
df = df_tmp

In [67]:
from datetime import datetime

now = datetime.now()
month = now.month
day = now.day
month_day = f'{month}.{day}'

if timing == 'am':
    out = 'Bear_断货改价_Hebe_上午 ' + month_day + '.xlsx'
else:
    out = 'Bear_断货改价_Hebe_下午 ' + month_day + '.xlsx'

df.to_excel('./' + out, 
            index=False ,
            startrow=1)

# 7.改格式

In [68]:
from openpyxl import load_workbook

In [69]:
wb_last = load_workbook(last)
ws_last = wb_last.worksheets[0]
ws_last

<Worksheet "Sheet1">

In [70]:
wb_out = load_workbook('./' + out)
ws_out = wb_out.worksheets[0]

In [71]:
coef = 1.125
ws_out.column_dimensions['A'].width = coef * 5.56
ws_out.column_dimensions['B'].width = coef * 18.69
ws_out.column_dimensions['C'].width = coef * 18.69
ws_out.column_dimensions['D'].width = coef * 6.88
ws_out.column_dimensions['E'].width = coef * 6.88
ws_out.column_dimensions['F'].width = coef * 9.88
ws_out.column_dimensions['G'].width = coef * 6.88
ws_out.column_dimensions['H'].width = coef * 9.88
ws_out.column_dimensions['I'].width = coef * 9.88
ws_out.column_dimensions['J'].width = coef * 7.04
ws_out.column_dimensions['K'].width = coef * 5.5
ws_out.column_dimensions['L'].width = coef * 5.5
ws_out.column_dimensions['M'].width = coef * 5.5
ws_out.column_dimensions['N'].width = coef * 5.5
ws_out.column_dimensions['O'].width = coef * 5.5
ws_out.column_dimensions['P'].width = coef * 5.5
ws_out.column_dimensions['Q'].width = coef * 5.5
ws_out.column_dimensions['R'].width = coef * 5.5
ws_out.column_dimensions['S'].width = coef * 5.5
ws_out.column_dimensions['T'].width = coef * 5.5
ws_out.column_dimensions['U'].width = coef * 5.5
ws_out.column_dimensions['V'].width = coef * 5.5
ws_out.column_dimensions['W'].width = coef * 5.5
ws_out.column_dimensions['X'].width = coef * 10.88
ws_out.column_dimensions['Y'].width = coef * 13
ws_out.column_dimensions['Z'].width = coef * 10.88
ws_out.column_dimensions['AA'].width = coef * 10.88
ws_out.column_dimensions['AB'].width = coef * 13
ws_out.column_dimensions['AC'].width = coef * 10.88
ws_out.column_dimensions['AD'].width = coef * 10.88
ws_out.column_dimensions['AE'].width = coef * 10.88
ws_out.column_dimensions['AF'].width = coef * 8.75
ws_out.column_dimensions['AG'].width = coef * 8.75

ws_out.row_dimensions[1].height = 25
ws_out.row_dimensions[2].height = 34
for i in range(3, ws_out.max_row+1):
    ws_out.row_dimensions[i].height = 16.5

In [72]:
from openpyxl.styles import Font, PatternFill
from openpyxl.styles import *
import datetime

In [73]:
orange ='00FFCC99'
blue_dark = 'FF8DB4E2'
green = 'FF92D050'
yellow = '00FFFF00'

now = datetime.datetime.now()
month = now.month
day = now.day
month_day = f'{month}/{day}'

ws_out['A1'].value = month_day
ws_out['A1'].font = Font(bold=True)

ws_out.merge_cells('B1:E1')

ws_out['G1'].value = '任务栏'

ws_out.merge_cells('H1:K1')

ws_out['O1'].value = '近一周销量'
ws_out.merge_cells('O1:V1')

ws_out['X1'].value = '10.5-J310.18-T6(日期最新在后面）'

In [74]:
ws_out['AA2'].fill = PatternFill(patternType='solid', fgColor=yellow)

for i in ws_out['A2:J2']:
    for j in i:
        j.fill = PatternFill(patternType='solid', fgColor=blue_dark)
        
for i in ws_out['K2:N2']:
    for j in i:
        j.fill = PatternFill(patternType='solid', fgColor=green)

for i in ws_out['O2:V2']:
    for j in i:
        j.fill = PatternFill(patternType='solid', fgColor=orange)
         
for i in ws_out['W2:Z2']:
    for j in i:
        j.fill = PatternFill(patternType='solid', fgColor=blue_dark)
        
for i in ws_out['AB2:AH2']:
    for j in i:
        j.fill = PatternFill(patternType='solid', fgColor=blue_dark)
        
for i in ws_out['M2:N2']:
    for j in i:
        j.value = datetime.datetime.strptime(str(j.value)[:17] + '01', '%Y-%m-%d %H:%M:%S')
        j.number_format = 'm/d;@'
        
for i in ws_out['P2:V2']:
    for j in i:
        j.value = datetime.datetime.strptime(str(j.value)[:17] + '01', '%Y-%m-%d %H:%M:%S')
        j.number_format = 'm/d;@'
#         j.fill = PatternFill(patternType='solid', fgColor=blue_light)

In [75]:
# for i in range(3, ws_out.max_row+1):
#     row = ws_out['AH'+str(i)].value
#     if row != None:
#         if timing == 'am':
#             color_C = ws_last['D'+str(row)].font.color
#             color_N = ws_last['Z'+str(row)].font.color
#             color_M = ws_last['Y'+str(row)].font.color
#             color_L = ws_last['X'+str(row)].font.color
#             color_K = ws_last['X'+str(row)].font.color

#         else:
#             color_C = ws_last['D'+str(row)].font.color
#             color_N = ws_last['AA'+str(row)].font.color
#             color_M = ws_last['Z'+str(row)].font.color
#             color_L = ws_last['Y'+str(row)].font.color
#             color_K = ws_last['X'+str(row)].font.color

#     else:
#         color_C = None
#         color_N = None
#         color_M = None
#         color_L = None
#         color_K = None
    
#     if color_C != None:
#         color_C = color_C.rgb
#     else:
#         color_C = None  
    
#     if color_N != None:
#         color_N = color_N.rgb
#     else:
#         color_N = None
    
#     if color_M != None:
#         color_M = color_M.rgb
#     else:
#         color_M = None

#     if color_L != None:
#         color_L = color_L.rgb
#     else:
#         color_L = None
    
#     if color_K != None:
#         color_K = color_K.rgb
#     else:
#         color_K = None
        

# #     print(i,color_C,color_N, color_M, color_L, color_K)

#     if type(color_C) == str:
#         ws_out['C'+str(i)].font = Font(color=color_C)
    
#     if type(color_N) == str:
#         ws_out['N'+str(i)].font = Font(color=color_N)
    
#     if type(color_M) == str:
#         ws_out['M'+str(i)].font = Font(color=color_M)

#     if type(color_L) == str:
#         ws_out['L'+str(i)].font = Font(color=color_L)
        
#     if type(color_K) == str:
#         ws_out['K'+str(i)].font = Font(color=color_K)


In [76]:

for i in range(3, ws_out.max_row+1):
    row = ws_out['AH'+str(i)].value
    if row != None:
        if timing == 'am':
            color_C = ws_last['C'+str(row)].font.color
            color_N = ws_last['M'+str(row)].font.color
            color_M = ws_last['L'+str(row)].font.color
            color_L = ws_last['K'+str(row)].font.color
            color_K = ws_last['K'+str(row)].font.color

        else:
            color_C = ws_last['C'+str(row)].font.color
            color_N = ws_last['N'+str(row)].font.color
            color_M = ws_last['M'+str(row)].font.color
            color_L = ws_last['L'+str(row)].font.color
            color_K = ws_last['K'+str(row)].font.color
            
    else:
        color_C = None
        color_N = None
        color_M = None
        color_L = None
        color_K = None
    
    if color_C != None:
        color_C = color_C.rgb
    else:
        color_C = None  
    
    if color_N != None:
        color_N = color_N.rgb
    else:
        color_N = None
    
    if color_M != None:
        color_M = color_M.rgb
    else:
        color_M = None

    if color_L != None:
        color_L = color_L.rgb
    else:
        color_L = None
    
    if color_K != None:
        color_K = color_K.rgb
    else:
        color_K = None
        

#     print(i,color_C,color_N, color_M, color_L, color_K)

    if type(color_C) == str:
        ws_out['C'+str(i)].font = Font(color=color_C)
    
    if type(color_N) == str:
        ws_out['N'+str(i)].font = Font(color=color_N)
    
    if type(color_M) == str:
        ws_out['M'+str(i)].font = Font(color=color_M)

    if type(color_L) == str:
        ws_out['L'+str(i)].font = Font(color=color_L)
        
    if type(color_K) == str:
        ws_out['K'+str(i)].font = Font(color=color_K)


In [77]:
from datetime import datetime
import time

In [78]:
wb_out.save('./' + out.replace('Bear_', 'color_Bear_'))
time.sleep(3)
wb_out.close()
print(datetime.now())

2023-12-04 17:30:58.952380
